In [4]:
import requests
from bs4 import BeautifulSoup
import json
import os,shutil
from time import sleep

hder_jd = {
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:65.0) Gecko/20100101 Firefox/65.0',
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
'Accept-Language': 'en-US,en;q=0.8,zh-CN;q=0.5,zh;q=0.3',
'Referer': 'https://www.jd.com/',
'DNT': '1',
'Connection': 'keep-alive',
'Upgrade-Insecure-Requests': '1',
'TE': 'Trailers',
}#https://blog.csdn.net/nuweike1879/article/details/108524524


In [3]:
def jd_get_detail_info(id):
    url='https://item.jd.com/'+id+'.html'
    info_dic={}#储存规格信息
    r=requests.get(url,headers=hder_jd)
    soup=BeautifulSoup(r.text, 'html.parser')
    info=soup.body.find('div',id='detail').find(attrs={'class':'Ptable'})#第一次使用id检索更准确,第二次用attrs检索是因为内容少

    for dl in info.find_all(attrs={'class':'clearfix'}):
        info_dic[dl.dt.text]=dl.find_all('dd')[-1].text#jd有的注释在真正的信息前面,所以选择最后一个该标签

    sleep(1)
    return info_dic

def jd_get_comment(id):
    url='https://club.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98&productId={}&score=0&sortType=5&page={}&pageSize=10&isShadowSku=0&rid=0&fold=1'
    #https://zhuanlan.zhihu.com/p/139753206
    #这篇文章讲解了从这里根据id和page获取评论信息
    comments=[]
    for page in range(1,11):#每页有10条评论信息
        r = requests.get(url.format(id,page), headers=hder_jd)
        data = r.text
        jd = json.loads(data.lstrip('fetchJSON_comment98(').rstrip(');'))
        data_list = jd['comments']
        #print("爬取了{}的第{}页评论信息".format(id,page))

        for data in data_list:
            comments.append(data['content'])

        sleep(1) #过于频繁的爬取会导致服务器封掉ip,所幸过不长时间又解封了

    return comments

def jd_init(goods):
    if os.path.exists(goods):
        shutil.rmtree(goods)
    os.mkdir(goods)
    
    ids=[]

    for page in range(1,3):#每页有30条id信息
        url = 'https://search.jd.com/Search?keyword={0}&enc=utf-8&wq={0}&page={1}'.format(goods,page)
        r = requests.get(url,headers=hder_jd)
        soup=BeautifulSoup(r.text, 'html.parser')
        for li in soup.body.find('div',id='J_goodsList').find_all('li'):
            ids.append(li.get('data-sku'))
        sleep(1)

    f=open(goods+"/id.txt",'w')
    for id in ids:
        f.write(id+'\n')
    f.close()

def jd_get_one_id(goods):
    if os.path.getsize(goods+'/id.txt')==0:
        return
    f=open(goods+'/id.txt','r')
    lines=f.readlines()
    f.close()
    
    id=lines[0].replace('\n','')
    info_dic=jd_get_detail_info(id)
    comments=jd_get_comment(id)
    
    lines.pop(0)
    f=open(goods+"/id.txt",'w')
    for line in lines:
        f.write(line)
    f.close()
    
    f=open(goods+"/comment.txt",'a')
    for comment in comments:
        f.write(comment.replace('\n', '').replace('\r', '')+'\n')#每个评论占且仅占一行
    f.close()
    
    f=open(goods+"/info.txt",'a')
    f.write(id+'\n')
    for (k,v) in info_dic.items():
        f.write(k+' '+v+'\n')
    f.close()   
    
    sleep(1)

In [ ]:
goods=''#填入希望爬的商品名字
jd_init(goods)#初始化,从search界面获取id信息

In [ ]:
while os.path.getsize(goods+'/id.txt')!=0:
    jd_get_one_id(goods)